In [75]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [86]:
#Crawling MRT Station names from Wiki
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations_by_planning_area').text
soup = BeautifulSoup(website_url, 'lxml')
table = soup.find('table', {'class':'wikitable sortable'})

#Crawling the Towns from Wiki
mrt_list_by_towns = dict({})
td_list = table.find_all('td')
keys = []
values = []

#Extracting the towns as keys
for index in range(0,len(td_list),6):
    keys.append(td_list[index].a.text)
    
#Extracting the stations associated with each Town
for index in range(1,len(td_list)+1,6):
    lst = []
    for i in range(len(td_list[index].find_all('a'))):
        lst.append(td_list[index].find_all('a')[i].text + ' MRT')
    values.append(lst)

#zipping values to populate the dict
for i in range(len(keys)):
    mrt_list_by_towns[keys[i]] = values[i]

#format into df and grouping by their key(towns)
town_stations = pd.DataFrame(columns=['Town', 'Station'])
for town in mrt_list_by_towns:
    for value in mrt_list_by_towns[town]:
        town_stations = town_stations.append({'Town': town, 'Station': value}, ignore_index=True)
town_stations

#Writing it to a csv
town_stations.to_csv("Town-Stations.csv", index=False)

,Town,Station
0,Ang Mo Kio,Yio Chu Kang MRT
1,Ang Mo Kio,Ang Mo Kio MRT
2,Bedok,Tanah Merah MRT
3,Bedok,Bedok MRT
4,Bedok,Kembangan MRT
...,...,...
137,Woodlands,Admiralty MRT
138,Woodlands,Woodlands North MRT
139,Woodlands,Woodlands South MRT
140,Yishun,Yishun MRT


In [44]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_Singapore_MRT_stations').text
soup = BeautifulSoup(website_url, 'lxml')
table = soup.find('table', {'class':'wikitable'})

#Table sorted by rows, enclosed in <tr>, so we will sort every row as list, and extract the 2nd element, and get the
#text from the second element. 
row_list = table.find_all('tr')

station_list = []


for i in range(3, len(row_list)):
    
    #third column, so search for third <td>
    row = row_list[i].find_all('td')
    try:
        station_name = row[2].text
        if row[2].find('i') == None:
            station_list.append(station_name + ' MRT')
    except:
        next 
        
#renaming any funny names
for index, name in enumerate(station_list):
    if 'Botanic' in (station_list[index]):
        station_list[index] = 'Botanic Gardens MRT'

station_list = (list(set(station_list)))
station_list.sort()
print(station_list)

['Admiralty MRT', 'Aljunied MRT', 'Ang Mo Kio MRT', 'Bartley MRT', 'Bayfront MRT', 'Beauty World MRT', 'Bedok MRT', 'Bedok North MRT', 'Bedok Reservoir MRT', 'Bencoolen MRT', 'Bendemeer MRT', 'Bishan MRT', 'Boon Keng MRT', 'Boon Lay MRT', 'Botanic Gardens MRT', 'Braddell MRT', 'Bras Basah MRT', 'Buangkok MRT', 'Bugis MRT', 'Bukit Batok MRT', 'Bukit Gombak MRT', 'Bukit Panjang MRT', 'Buona Vista MRT', 'Caldecott MRT', 'Canberra MRT', 'Cashew MRT', 'Changi Airport MRT', 'Chinatown MRT', 'Chinese Garden MRT', 'Choa Chu Kang MRT', 'City Hall MRT', 'Clarke Quay MRT', 'Clementi MRT', 'Commonwealth MRT', 'Dakota MRT', 'Dhoby Ghaut MRT', 'Dover MRT', 'Downtown MRT', 'Esplanade MRT', 'Eunos MRT', 'Expo MRT', 'Farrer Park MRT', 'Farrer Road MRT', 'Fort Canning MRT', 'Geylang Bahru MRT', 'Gul Circle MRT', 'HarbourFront MRT', 'Haw Par Villa MRT', 'Hillview MRT', 'Holland Village MRT', 'Hougang MRT', 'Jalan Besar MRT', 'Joo Koon MRT', 'Jurong East MRT', 'Kaki Bukit MRT', 'Kallang MRT', 'Kembangan M

In [49]:
latitudes = []
longitudes = []

def getGeocodes(location_list): #location_names passed as a list
    for location in location_list:
        query_string = 'https://developers.onemap.sg/commonapi/search?searchVal=' + str(location) + '&returnGeom=Y&getAddrDetails=N&pageNum=1'
        result = requests.get(query_string)
        data = json.loads(result.content)
    
        latitudes.append(float(data['results'][0]['LATITUDE']))
        longitudes.append(float(data['results'][0]['LONGITUDE']))
    
    dict = {'Location': location_list, 'Latitude': latitudes, 'Longitudes': longitudes}
    location_geocodes = pd.DataFrame(dict)
    return(location_geocodes)

In [50]:
#Creating a CSV for MRT Stations and their geocodes
#getGeocodes(station_list).to_csv('mrt_stations_geocodes.csv', index=False)